In [1]:
import tenseal as ts
import utils

# generate keys

In [2]:
context = ts.Context(
    ts.SCHEME_TYPE.CKKS,
    poly_modulus_degree=8192,
    coeff_mod_bit_sizes=[60, 40, 40, 60]
)

In [3]:
context.generate_galois_keys()
context.global_scale = 2 ** 40

In [4]:
secret_context = context.serialize(save_secret_key=True)
utils.write_data("keys/secret.txt", secret_context)

In [5]:
context.make_context_public() # drops private key
public_context = context.serialize()
utils.write_data("keys/public.txt", public_context)

# encrypt

In [6]:
context = ts.context_from(utils.read_data("keys/secret.txt"))

In [7]:
import pandas as pd
from sklearn.preprocessing import OneHotEncoder, LabelEncoder

encoder = OneHotEncoder(
    sparse_output=False,
    dtype=int
)

df = pd.read_csv("../data/votes.csv")

candidatos = encoder.fit_transform(df[["Candidates"]])

candidatos

array([[0, 0, 1, ..., 0, 0, 0],
       [0, 0, 0, ..., 1, 0, 0],
       [0, 0, 0, ..., 0, 1, 0],
       ...,
       [0, 0, 0, ..., 1, 0, 0],
       [1, 0, 0, ..., 0, 0, 0],
       [0, 0, 0, ..., 0, 1, 0]])

In [8]:
for i, candidato in enumerate(candidatos):
    candidatos_enc = ts.ckks_vector(context, candidato)
    if i == 0:
        utils.write_data("outputs/candidatos_enc.txt", candidatos_enc.serialize())
    else:
        utils.write_data_append("outputs/candidatos_enc.txt", candidatos_enc.serialize())

# decryption

In [9]:
m_proto = utils.read_data("outputs/soma.txt")
m = ts.lazy_ckks_vector_from(m_proto)
m.link_context(context)

In [10]:
votos_finais = m.decrypt()

# Get the mapping of numbers (columns) to candidate names
column_mapping = dict(enumerate(encoder.categories_[0]))

# Display the final votes
for number, votes in enumerate(votos_finais):
    print(f"{column_mapping[number]}: {votes:.0f}")

Branco: 95
Emanuel: 100
Filipa: 398
Maria: 49
Marta: 213
Nulo: 105
Nuno: 39
